This notebook is currently in progress

# GEE Aster Preprocessing

## Step 1: Create a google cloud project

Follow this link to create a google could project: https://cloud.google.com/resource-manager/docs/creating-managing-projects

## Step 2: Installation


In [4]:
%pip install git+https://github.com/Mining-for-the-Future/ASTER_preprocessing.git@Authentification_issue

  Cloning https://github.com/Mining-for-the-Future/ASTER_preprocessing.git (to revision Authentification_issue) to c:\users\eweav\appdata\local\temp\pip-req-build-y88ebvvo
  Resolved https://github.com/Mining-for-the-Future/ASTER_preprocessing.git to commit 17dfa729cc4db34d5a402ccd5d6689dbc65a77f6
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/Mining-for-the-Future/ASTER_preprocessing.git 'C:\Users\eweav\AppData\Local\Temp\pip-req-build-y88ebvvo'
  Running command git checkout -b Authentification_issue --track origin/Authentification_issue
  branch 'Authentification_issue' set up to track 'origin/Authentification_issue'.
  Switched to a new branch 'Authentification_issue'


## Step 3: Importing Libraries

In [2]:
import ASTER_preprocessing

In [5]:
from ASTER_preprocessing import preprocessing


Successfully saved authorization token.


In [ ]:
# %pip install geemap

In [6]:
import geemap

## Step 4: Google Earth Engine Authentication

In [10]:
ee.Authenticate()
ee.Initialize(project="")

## Step 5: Preprocess ASTER Imagery Using a Drawn Polygon

In [7]:
# Create an interactive map
Map = geemap.Map()

# Add drawing tools to the map
# Map.addDrawingTools()

# Display the map
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [11]:
geometry = Map.user_roi

image = preprocessing.aster_preprocessing(geom = geometry)['imagery']
Map.addLayer(image)

NameError: name 'ee_i' is not defined

In [8]:
# Get the last drawn feature
geometry = Map.user_roi

# Check if a geometry was drawn and it's a Polygon
if geometry is not None and geometry.geometry().type().getInfo() == 'Polygon':
    # Run the preprocessing
    result = preprocessing.aster_preprocessing(geometry.geometry())
    # `result` is a dictionary containing the preprocessed image and metadata
else:
    print("Please draw a polygon on the map.")

AttributeError: 'Geometry' object has no attribute 'geometry'